In [102]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [103]:
from bs4 import BeautifulSoup
import requests, re
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'http://operabase.com/visual.cgi?is=composer&max=5000&break=25'
b_url = 'http://operabase.com/visual.cgi?'

In [109]:
r  = requests.get(base_url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")
results = soup.find('ul').next_sibling.find('tbody').find_all('tr')
composers = []


def remove_dates(text):
    try:
        text = text.replace(re.search('\((.*?)\)', text).group(0), '')   
    except:
        None
    return text

for result in results:
    rows = result.find_all('tr')
    for row in rows:
        comp_name = row.find('a').get_text()
        comp_url  = b_url + row.find('a').get('href')        
        if comp_name=='Prokofiev':
            comp_url = comp_url + ',S'
#         elif comp_name=='Suppe':

        r  = requests.get(comp_url, headers=headers)
        soup = BeautifulSoup(r.text, "html5lib")
        
#         works = soup.find('table').find('tbody').find_all('table')[2]
        try:
            works = soup.find(text=re.compile(r'statistics')).parent.parent.next_sibling
            works = works.find('table').find('table').find_all('tr')    

            comp_info = soup.find(text=re.compile(r'From the composers database')).parent.parent.next_sibling
            comp_info = comp_info.find('table').find('a').parent
            comp_full_name = comp_info.find('a').get_text().encode(encoding='iso-8859-1').decode('utf-8').strip()        
            comp_full_name = remove_dates(comp_full_name)
#             print(comp_full_name)
            if len(comp_full_name.split(', '))>1:
                comp_full_name = comp_full_name.split(', ')[1].strip() + ' ' + comp_full_name.split(', ')[0].strip()
            else:
                comp_full_name = comp_full_name
            comp_dates = comp_dates = comp_info.find_all('font')[1].get_text()
            
            for work in works:
                composer = []                 
                work_name = work.find_all('td')[4].get_text()#.encode(encoding='iso-8859-1').decode('utf-8').strip()
                composer.append(comp_name)
                composer.append(comp_full_name)
                composer.append(comp_dates)
                composer.append(comp_url)            
                composer.append(work_name)
#                 print(composer)
                composers.append(composer)
        except:
            print(comp_url)

http://operabase.com/visual.cgi??lang=en&is=opera&by=Suppe
http://operabase.com/visual.cgi??lang=en&is=opera&by=Zykan
http://operabase.com/visual.cgi??lang=en&is=opera&by=Schaffer
http://operabase.com/visual.cgi??lang=en&is=opera&by=Albrecht
http://operabase.com/visual.cgi??lang=en&is=opera&by=Harris,J


In [67]:
# composers[0]

In [110]:
#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['composer', 'complete_name', 'dates', 'uri', 'work']
df = pd.DataFrame(composers, columns=headers)
df.to_csv('operabase_works_composers.csv')

In [68]:
# df.head()

In [111]:
#database connection settings
import pandas as pd
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)


df = pd.read_csv('operabase_works_composers.csv', index_col=0)
df = df[['composer', 'complete_name', 'dates', 'uri']]
df = df.drop_duplicates()

for idx,row in df.iterrows():
    search = row['composer'].split(',')[0].lower().replace("'", "''")
    cursor = conn.cursor()  
    query = "SELECT name as composer, id FROM composers WHERE lower(unaccent(name)) like '%" + search + "%'"
    cursor.execute(query)
    c_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'id'])
    if c_df.shape[0]==0:
        insert = 'INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('
        values = "'" + row['complete_name'].replace("'", "''") + "', '" +  row['dates'].replace("'", "''") + "', '" + row['uri'].replace("'", "''") + "'); "
        commit = 'COMMIT;'    
        print(insert + values + commit)
#         print('Results for ' + row['composer'] + ': ' + str(c_df.shape[0]))

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Modest Petrovich Musorgsky', '(1839–1881)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Musorgsky'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Sergey Vasil''yevich Rakhmaninov', '(1873–1943)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Rakhmaninov'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Stuart MacRae', '(b.1976)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=MacRae'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Russell Hepplewhite', '(b.1982)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Hepplewhite'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Calliope Tsoupaki', '(b.1963)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Tsoupaki'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Semyon Stepanovich Gulak-Artemovsky', '(1813–1873)', 'http://operabase.com/visual.c

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Christian Henking', '(b.1961)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Henking'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Antoni Lliteres', '(1673–1747)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Lliteres'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Dmitry Smolsky', '(b.1937)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Smolsky'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Gérard Pesson', '(b.1958)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Pesson'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Alessandro Stradella', '(1639–1682)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Stradella'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Ivo Tijardović', '(1895–1976)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Tijardovic'); COMMIT;
INSERT INTO

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Onutė Narbutaitė', '(b.1956)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Narbutaite'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Jonas Tamulionis', '(b.1949)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Tamulionis'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Mats Larsson Gothe', '(b.1965)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Larsson Gothe'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Tevfik Akbaşli', '(b.1962)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Akbasli'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Harry Partch', '(1901–1974)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Partch'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Frédéric Verrieres', '(b.1968)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Verrieres'); COMMIT;
INSERT I

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Johannes Maria Staud', '(b.1974)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Staud'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Sven Helbig', '
 (personal/ official website)  (email address is available to subscribing opera companies) German composer and director. compositeur et metteur en scène allemand. compositore e regista tedesco. deutscher Komponist und Regisseur.', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Helbig'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Gunnar Thordarson', '(b.1945)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Thordarson'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Francesco Gardi', '(1760–1810)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Gardi'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Henning Sommerro', '(b.1952)', 'http://operabase.com/visual.cgi??lan

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Carlo Boccadoro', '(b.1963)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Boccadoro'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Cristian Carrara', '(b.1977)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Carrara'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Kh Bilegjargal', '(1954–2008)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Bilegjargal'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Ts Natsagdorj', '(b.1951)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Natsagdorj'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Vladimir Martynov', '(b.1946)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Martynov'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Chico Mello', '(b.1957)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Mello'); COMMIT;
INSERT INTO COMPOSERS

...

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Pierantonio Tasca', '(1858–1934)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Tasca'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Martijn Padding', '(b.1956)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Padding'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Dan Dediu', '(b.1967)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Dediu'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Morse Thomas', '
http://www.musicsalesclassical.com/composer/short-bio/Thomas-Morse (other) American composer. compositeur amÃ©ricain. compositore americano. amerikanischer Komponist.', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Morse'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('James Reynolds', '(b.1953)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Reynolds'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TE

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Christian Lauba', '(b.1952)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Lauba'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Mirela Ivičević', '(b.1980)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Ivicevic'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Judit Varga', '(b.1970)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Varga'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Samu Gryllus', '(b.1976)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Gryllus'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Daníel Bjarnason', '(b.1979)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Bjarnason'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Giacinto Scelsi', '(1905–1988)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Scelsi'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKI

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('León Biriotti', '(b.1929)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Biriotti'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Tokhtasin Jalilov', '
http://sanat.orexca.com/eng/3-4-07/tokhtasin_jalilov.shtml (other) Uzbek musician and composer. musicien et compositeur ouzbek. musicista e compositore uzbeco. usbekischer Musiker und Komponist.', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Jalilov'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Hanne Raffnsøe', '
  composer. compositrice. compositrice.  Komponistin.', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Raffnsoe'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Lucie Treacher', '
 (personal/ official website)  composer, songwriter and singer. compositrice, songwriter et chanteuse. compositrice, songwriter e cantante.  Komponistin, songwriter und Sängerin.', 'http://operabase.com

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Wynton Marsalis', '(b.1961)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Marsalis'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Dominique Pauwels', '
  composer. compositeur. compositore.  Komponist.', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Pauwels'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Ture Rangström', '(1884–1947)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Rangstrom'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Giacomo Tritto', '(1733–1824)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Tritto'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Garbis Aprikian', '(b.1926)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Aprikian'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('James Hullick', '
 (personal/ official website) Australian composer, sound, arti

INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Vittorio Gnecchi', '(1876–1954)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Gnecchi'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Alessandro Melani', '(1639–1703)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Melani,A'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Venanzio Rauzzini', '(1746–1810)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Rauzzini'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Michele Varriale', '(b.1976)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Varriale'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Angelo Bellisario', '(b.1932)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Bellisario'); COMMIT;
INSERT INTO COMPOSERS (NAME, WIKIPEDIA_TEXT, URI) VALUES ('Giovanni Bertelli', '(b.1980)', 'http://operabase.com/visual.cgi??lang=en&is=opera&by=Bertelli'); COMMIT;
IN

In [143]:
#database connection settings
import pandas as pd
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)


df = pd.read_csv('operabase_works_composers.csv', index_col=0)

for idx,row in df.iterrows():
    
    search_c = row['composer'].split(',')[0].lower().replace("'", "''").strip()
    search_c = search_c.replace('musorgsky', 'mussorgsky')
    search_c = search_c.replace('rakhmaninov', 'rachmaninoff')    
    
    search_w = row['work'].split(',')[0].lower()
    search_w = search_w.replace("'", "''")
    search_w = search_w.replace('(adaptation)', '')
    search_w = search_w.replace('(adpatation)', '')    
    search_w = search_w.replace('(reduction)', '') 
    search_w = search_w.replace('[c]', '')        
    search_w = search_w.strip()
    search_w = search_w.split(' ')[-1]
    
    save_work = row['work'].replace("'", "''")
    save_work = save_work.replace('(adaptation)', '')
    save_work = save_work.replace('(adpatation)', '')    
    save_work = save_work.replace('(reduction)', '') 
    save_work = save_work.replace('[c]', '')        
    save_work = save_work.strip()
    
    cursor = conn.cursor()  
    query  = "SELECT w.name as work, w.id FROM composers c INNER JOIN works w ON w.composer_id = c.id WHERE lower(unaccent(c.name)) like '%" + search_c + "%' and lower(unaccent(w.aka)) like '%" + search_w + "%'"
    cursor.execute(query)
    w_df   = pd.DataFrame(cursor.fetchall(), columns=['name', 'id'])
    
    if w_df.shape[0]==0:
        
        cursor      = conn.cursor()          
        query       = "SELECT name as composer, id FROM composers WHERE lower(unaccent(name)) like '%" + search_c + "%' LIMIT 1"
        cursor.execute(query)
        c_df        = pd.DataFrame(cursor.fetchall(), columns=['name', 'id'])
        composer_id = c_df.iloc[0]['id']
        
        cursor = conn.cursor()          
        query  = "SELECT max(id) FROM works"
        cursor.execute(query)
        uri_df = pd.DataFrame(cursor.fetchall(), columns=['id'])        
        uri    = 'W' + str(uri_df.iloc[0]['id'] + 1)        
    
        insert = 'INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('
        values = "'" + save_work + "', '" + uri + "', " + str(composer_id) + ", '" + save_work + "'); "
        commit = 'COMMIT;'    
        print(insert + values + commit)

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Le notti delle streghe', 'W21779', 763, 'Le notti delle streghe'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('La Signora Doria', 'W21779', 400, 'La Signora Doria'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Koningin van de Nacht', 'W21779', 1037, 'Koningin van de Nacht'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Grande Messe en ut mineur', 'W21779', 1037, 'Grande Messe en ut mineur'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Messe c-Moll', 'W21779', 1037, 'Messe c-Moll'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Mozart-Pasticcio', 'W21779', 1037, 'Mozart-Pasticcio'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('La petite messe solennelle', 'W21779', 408, 'La petite messe solennelle'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Le nozze di Teti e di Peleo', 'W21779', 408, 'L

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Das Waldmadchen', 'W21779', 1074, 'Das Waldmadchen'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Die Wolfsschlucht', 'W21779', 1074, 'Die Wolfsschlucht'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Castor et Pollux', 'W21779', 547, 'Castor et Pollux'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Zoroastre', 'W21779', 547, 'Zoroastre'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Daphnis et Egle', 'W21779', 547, 'Daphnis et Egle'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Les Fetes d''Hebe', 'W21779', 547, 'Les Fetes d''Hebe'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('La Princesse de Navarre', 'W21779', 547, 'La Princesse de Navarre'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Les Fetes de Polymnie', 'W21779', 547, 'Les Fetes de Polymnie'); COMMIT;
INSERT INTO WORKS (NAME, URI

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Das Lange Weihnachtsmahl', 'W21779', 832, 'Das Lange Weihnachtsmahl'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Die Passagierin', 'W21779', 760, 'Die Passagierin'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Wir gratulieren!', 'W21779', 760, 'Wir gratulieren!'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Swanhunter', 'W21779', 592, 'Swanhunter'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Flight', 'W21779', 592, 'Flight'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Monster in the Maze', 'W21779', 592, 'The Monster in the Maze'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Mansfield Park', 'W21779', 592, 'Mansfield Park'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Enchanted Pig', 'W21779', 592, 'The Enchanted Pig'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Zaporozhets za Dunayem', 'W21779', 2826, 'Zaporozhets za Dunayem'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Markheim', 'W21779', 157, 'Markheim'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Slow Dusk', 'W21779', 157, 'Slow Dusk'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Prince of Players', 'W21779', 157, 'Prince of Players'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Rusalka', 'W21779', 2827, 'Rusalka'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Stone Guest', 'W21779', 2827, 'The Stone Guest'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('La finta tedesca', 'W21779', 561, 'La finta tedesca'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Demetrio', 'W21779', 561, 'Demetrio'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Lucio Papirio', 'W21779', 561, '

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Die Gespenstersonate', 'W21779', 106, 'Die Gespenstersonate'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('I am your opus', 'W21779', 106, 'I am your opus'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Papageno spielt auf der Zauberflote', 'W21779', 2831, 'Papageno spielt auf der Zauberflote'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Die Sternstunde des Josef Bieder', 'W21779', 2831, 'Die Sternstunde des Josef Bieder'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Der arme Matrose', 'W21779', 193, 'Der arme Matrose'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('L''Enlevement d''Europe', 'W21779', 193, 'L''Enlevement d''Europe'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Bear', 'W21779', 1097, 'The Bear'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Facade', 'W21779', 1097, 'Fa

...

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Lan Hua Hua', 'W21779', 3168, 'Lan Hua Hua'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Krakatit', 'W21779', 3169, 'Krakatit'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('No Man', 'W21779', 3170, 'No Man'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Refidim Junction', 'W21779', 1040, 'Refidim Junction'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Max und Moritz', 'W21779', 680, 'Max und Moritz'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Bonnie und Clyde', 'W21779', 3171, 'Bonnie und Clyde'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Orlando', 'W21779', 3172, 'Orlando'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Copernicus', 'W21779', 1264, 'Copernicus'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Rosen der Liebe', 'W21779', 3173, 'Rosen der Liebe'); CO

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('La Lettre des sables', 'W21779', 3216, 'La Lettre des sables'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Colomba', 'W21779', 2128, 'Colomba'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Eine klitzikleine Sache', 'W21779', 3217, 'Eine klitzikleine Sache'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Love', 'W21779', 3218, 'Love'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Two Women', 'W21779', 3219, 'Two Women'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Brodre', 'W21779', 3220, 'Brodre'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Chants du Capricorne', 'W21779', 3221, 'Chants du Capricorne'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('qui non c''e perche', 'W21779', 3222, 'qui non c''e perche'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Il suono giallo

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Poor Folk', 'W21779', 3265, 'Poor Folk'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Three Musketeers', 'W21779', 1522, 'The Three Musketeers'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Raoul Wallenberg, saknad', 'W21779', 3266, 'Raoul Wallenberg, saknad'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Der Steppenwolf', 'W21779', 3267, 'Der Steppenwolf'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Silver Birch', 'W21779', 3268, 'Silver Birch'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Heart of Darkness', 'W21779', 2753, 'Heart of Darkness'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Frankenstein', 'W21779', 677, 'Frankenstein'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('A New Kind of Fallout', 'W21779', 2323, 'A New Kind of Fallout'); COMMIT;
INSERT INTO WORKS (NAME, URI,

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Unicamente la verdad', 'W21779', 3316, 'Unicamente la verdad'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Atzimba', 'W21779', 3317, 'Atzimba'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('All About My Family', 'W21779', 3318, 'All About My Family'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Young Wife', 'W21779', 3319, 'The Young Wife'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Zamek na Czorsztynie', 'W21779', 645, 'Zamek na Czorsztynie'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Yarema and Danilo', 'W21779', 3320, 'Yarema and Danilo'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Alcide', 'W21779', 3321, 'Alcide'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Ugly Duckling', 'W21779', 3322, 'The Ugly Duckling'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) V

INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Afternoon of Love', 'W21779', 3368, 'The Afternoon of Love'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Madame Buffault', 'W21779', 3369, 'Madame Buffault'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('L''histoire du petit tailleur', 'W21779', 3370, 'L''histoire du petit tailleur'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Cataclysm', 'W21779', 3371, 'Cataclysm'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Scaevola', 'W21779', 3372, 'Scaevola'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('The Garden', 'W21779', 367, 'The Garden'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Sukanya', 'W21779', 3373, 'Sukanya'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Funeral Play', 'W21779', 3374, 'Funeral Play'); COMMIT;
INSERT INTO WORKS (NAME, URI, COMPOSER_ID, AKA) VALUES ('Serial Sev

In [ ]:
cursor = conn.cursor()  
query = "SELECT w.name as composer, w.id FROM composers c INNER JOIN WORKS w ON w.composer_id = c.id WHERE lower(unaccent(w.name)) like '%" + search + "%'"
cursor.execute(query)
c_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'id'])
